In [13]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
import re
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Embedding
from keras.layers import Bidirectional
from tensorflow.keras.preprocessing.text import one_hot
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Load data
df = pd.read_csv('./data/product_df.csv')
df = df[['Star Rating', 'Comment']]

In [3]:
def label_sentiment(rating):
    if rating in [1, 2]:
        return '0'
    elif rating == 3:
        return '1'
    elif rating in [4, 5]:
        return '2'
    else:
        return '3'  # Nếu có xếp hạng nằm ngoài khoảng 1-5

# Gắn nhãn cảm xúc cho mỗi đánh giá
df['Sentiment'] = df['Star Rating'].apply(label_sentiment)

# Hiển thị 5 hàng đầu tiên của dataframe với cột sentiment mới
print(df.head())

   Star Rating                                            Comment Sentiment
0            4                      Điện thoại này dùng rất thích         2
1            4                               sử dụng thấy cũng ok         2
2            2                      Bảo hành ít quá, chỉ 12 tháng         0
3            5                             Sản phẩm mượt, chạy êm         2
4            3  Cho mình hỏi muốn khởi động lại máy hay tắt ng...         1


In [4]:
# Kiểm tra và loại bỏ giá trị khuyết thiếu
print(df.isnull().sum())
df = df.dropna(subset=['Comment'])
# Kiểm tra và loại bỏ dữ liệu trùng lặp
duplicate_comments = df[df.duplicated(['Comment'])]
print("Các dòng dữ liệu trùng lặp trong cột 'Comment':")
print(duplicate_comments)
df = df.drop_duplicates(['Comment'])
print("Shape after dropping duplicates:", df.shape)

Star Rating    0
Comment        0
Sentiment      0
dtype: int64
Các dòng dữ liệu trùng lặp trong cột 'Comment':
      Star Rating                                   Comment Sentiment
48              4                                   Rất tốt         2
76              5                                    Rất ok         2
107             2                             pin tụt nhanh         0
135             5                              sản phẩm tốt         2
173             2                             hao pin nhanh         0
177             5                              sản phẩm tốt         2
181             5                                    Rất ok         2
183             5                              sản phẩm tốt         2
192             5                               sản phẩm ok         2
205             5                               máy dùng ok         2
225             4                                       Tốt         2
227             5                               

In [5]:
# Chuẩn hóa và làm sạch văn bản
def remove_special_characters(text):
    # Loại bỏ các ký tự đặc biệt, giữ lại chữ cái, số, và các dấu câu
    return re.sub(r'[^a-zA-ZÀ-ỹà-ỹ0-9\s]', '', text)

def to_lowercase(text):
    # Chuyển đổi văn bản về chữ thường
    return text.lower()

def normalize_text(text):
    text = remove_special_characters(text)
    text = to_lowercase(text)
    return text

df['Comment'] = df['Comment'].apply(normalize_text)
print(df.head())

   Star Rating                                            Comment Sentiment
0            4                      điện thoại này dùng rất thích         2
1            4                               sử dụng thấy cũng ok         2
2            2                       bảo hành ít quá chỉ 12 tháng         0
3            5                              sản phẩm mượt chạy êm         2
4            3  cho mình hỏi muốn khởi động lại máy hay tắt ng...         1


In [6]:
max_features = 20000  # Only consider the top 20k words
maxlen = 200

In [7]:
phrases = df['Comment'].tolist()
vocab_size = 50000
encoded_phrases = [one_hot(d, vocab_size) for d in phrases]
df['Comment'] = encoded_phrases

In [8]:
label_encoder = LabelEncoder()
df['Sentiment'] = label_encoder.fit_transform(df['Sentiment'])

In [9]:
# Chia dữ liệu thành đặc trưng (X) và nhãn (y)
X = df['Comment']
y = df['Sentiment']

In [10]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [11]:
x_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(X_val, maxlen=maxlen)

In [14]:
model = Sequential()
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
model.add(inputs)
model.add(Embedding(50000, 128))
# Add 2 bidirectional LSTMs
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
# Add a classifier
model.add(Dense(3, activation="sigmoid"))
#model = keras.Model(inputs, outputs)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, None, 128)      │     6,400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, None, 128)      │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,598,019 (25.17 MB)

 Trainable params: 6,598,019 (25.17 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_val, y_val))

Epoch 1/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 33s 512ms/step - accuracy: 0.5596 - loss: 0.9673 - val_accuracy: 0.6333 - val_loss: 0.8224
Epoch 2/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 15s 384ms/step - accuracy: 0.6954 - loss: 0.6921 - val_accuracy: 0.7300 - val_loss: 0.6736
Epoch 3/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 13s 354ms/step - accuracy: 0.8089 - loss: 0.4433 - val_accuracy: 0.7433 - val_loss: 0.6089
Epoch 4/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 16s 424ms/step - accuracy: 0.9037 - loss: 0.2818 - val_accuracy: 0.7567 - val_loss: 0.7142
Epoch 5/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 15s 397ms/step - accuracy: 0.9555 - loss: 0.1725 - val_accuracy: 0.7600 - val_loss: 0.7098
Epoch 6/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 21s 411ms/step - accuracy: 0.9716 - loss: 0.1124 - val_accuracy: 0.7367 - val_loss: 0.9528
Epoch 7/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 21s 427ms/step - accuracy: 0.9611 - loss: 0.1309 - val_accuracy: 0.7400 - val_loss: 0.9489
Epoch 8/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 16s 421ms/step - accuracy: 0.9611 - loss: 0.1111 - val_accu

In [20]:
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val))

Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 26s 668ms/step - accuracy: 0.9981 - loss: 0.0049 - val_accuracy: 0.7500 - val_loss: 1.5867
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 19s 491ms/step - accuracy: 0.9983 - loss: 0.0062 - val_accuracy: 0.7367 - val_loss: 1.6043
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 30s 792ms/step - accuracy: 0.9834 - loss: 0.0481 - val_accuracy: 0.7533 - val_loss: 1.4444
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 25s 364ms/step - accuracy: 0.9919 - loss: 0.0226 - val_accuracy: 0.7567 - val_loss: 1.4002
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 14s 371ms/step - accuracy: 0.9954 - loss: 0.0139 - val_accuracy: 0.7633 - val_loss: 1.4617
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 13s 352ms/step - accuracy: 0.9968 - loss: 0.0083 - val_accuracy: 0.7500 - val_loss: 1.5486
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 15s 385ms/step - accuracy: 0.9967 - loss: 0.0125 - val_accuracy: 0.7567 - val_loss: 1.4913
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 20s 522ms/step - accuracy: 0.9978 - loss: 0.0118 - val_accu

In [17]:
# Các thông số đã sử dụng trong quá trình huấn luyện
vocab_size = 50000
maxlen = 200
# Chuỗi đánh giá mới cần dự đoán
new_comment = "Rất tệ"

# Chuẩn hóa và mã hóa văn bản
normalized_comment = normalize_text(new_comment)
encoded_comment = one_hot(normalized_comment, vocab_size)
padded_comment = pad_sequences([encoded_comment], maxlen=maxlen)

# Dự đoán
prediction = model.predict(padded_comment)
sentiment_label = np.argmax(prediction)

# Gắn nhãn cảm xúc
if sentiment_label == 0:
    print('Cảm xúc: Tiêu cực')
elif sentiment_label == 1:
    print('Cảm xúc: Trung tính')
elif sentiment_label == 2:
    print('Cảm xúc: Tích cực')
else:
    print('Không xác định')

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Cảm xúc: Tiêu cực


In [18]:
# Đánh giá mô hình trên tập kiểm tra
_, accuracy = model.evaluate(x_val, y_val)
print(f'Accuracy trên tập kiểm tra: {accuracy:.2f}')


10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 241ms/step - accuracy: 0.7418 - loss: 1.5306
Accuracy trên tập kiểm tra: 0.75


In [19]:
from sklearn.metrics import classification_report, confusion_matrix

# Dự đoán trên tập kiểm tra
y_pred = model.predict(x_val)
y_pred_labels = np.argmax(y_pred, axis=1)

# Đánh giá mô hình
print(classification_report(y_val, y_pred_labels))


10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 118ms/step
              precision    recall  f1-score   support

           0       0.73      0.78      0.75        86
           1       0.43      0.42      0.43        48
           2       0.85      0.83      0.84       166

    accuracy                           0.75       300
   macro avg       0.67      0.68      0.67       300
weighted avg       0.75      0.75      0.75       300



In [ ]:
model.save('sentiment_model.h5')